In [1]:
import gym
import pybullet
import pybullet_envs
from gym import wrappers
from datetime import datetime

# DR TRPO related files
from train_helper import *
from value import NNValueFunction
from utils import Logger
from dr_policy import DRPolicyKL, DRPolicyWass

In [ ]:
#########################       Discrete State Space - KL DR Policy     ######################### 
# e.g. 'Taxi-v3', 'Roulette-v0', 'NChain-v0', 'FrozenLake-v0', 'CliffWalking-v0', 'FrozenLake8x8-v0'

env_name = 'CliffWalking-v0'
pybullet.connect(pybullet.DIRECT)
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyKL(sta_num, act_num)
val_func = NNValueFunction(1, 3)
gamma = 0.8
lam = 1
total_eps = 100
batch_eps = 3
logger = Logger(logname=env_name + '_DR-KL_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        policy.update(observes, actions, advantages)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

In [ ]:
#########################       Discrete State Space - Wasserstein DR Policy     ######################### 
# e.g. 'Taxi-v3', 'Roulette-v0', 'NChain-v0', 'FrozenLake-v0', 'CliffWalking-v0', 'FrozenLake8x8-v0'

env_name = 'Taxi-v3'
pybullet.connect(pybullet.DIRECT)
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyWass(sta_num, act_num)
val_func = NNValueFunction(1, 3)
gamma = 0.8
lam = 1
total_eps = 5000
batch_eps = 100
logger = Logger(logname=env_name + '_DR-Wass_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        policy.update(observes, actions, advantages, disc_freqs)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 3, h2: 3, h3: 5, lr: 0.00577
***** Episode 100, Mean Return = -773.1, Mean Discounted Return = -20.6 *****
ExplainedVarNew: -5.2e-09
ExplainedVarOld: -1.48e-05
ValFuncLoss: 54.5


***** Episode 200, Mean Return = -714.2, Mean Discounted Return = -17.4 *****
ExplainedVarNew: -2.55e-08
ExplainedVarOld: -2.47e-09
ValFuncLoss: 59.3


***** Episode 300, Mean Return = -639.7, Mean Discounted Return = -15.9 *****
ExplainedVarNew: -0.000499
ExplainedVarOld: -1.7e-08
ValFuncLoss: 61.7


***** Episode 400, Mean Return = -561.1, Mean Discounted Return = -16.0 *****
ExplainedVarNew: -0.000226
ExplainedVarOld: -0.000438
ValFuncLoss: 63.3


***** Episode 500, Mean Return = -495.9, Mean Discounted Return = -15.1 *****
ExplainedVarNew: -1.36e-05
ExplainedVarOld: -0.000159
ValFuncLoss: 58.3


***** Episode 600, Mean Return = -399.2, Mean Discounted Return = -13.2 *****
ExplainedVarNew: -5.95e-06
ExplainedVarOld: -3.54e-06
ValFuncLoss: 60.4


***** Episode 700, Mean Return = -360.3, 